In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder

import sys
import imp

sys.path.append('../src/')
import data.reader as dataReader
import utils.utils as thesisUtils
import similarities.cosine as thesisCosineSimilarity
import vocabulary.vocabulary as thesisVocabulary
import features.model_features as thesisModelFeatures
import data.corpus_stats as thesisCorpusStats
import features.factory as thesisFactoryFeatures

imp.reload(dataReader)
imp.reload(thesisUtils)
imp.reload(thesisCorpusStats)
imp.reload(thesisVocabulary)
imp.reload(thesisModelFeatures)
imp.reload(thesisCosineSimilarity)
imp.reload(thesisFactoryFeatures)

<module 'features.factory' from '../src/features/factory.py'>

In [19]:
london_corpus = dataReader.CorpusByNewLine.london()
zwickau_corpus = dataReader.CorpusByNewLine.zwickau()
burchard_corpus = dataReader.BurchardCorpus(london_corpus, zwickau_corpus)

london_leftovers = dataReader.LeftoversCorpus(london_corpus, zwickau_corpus)
zwickau_leftovers = dataReader.LeftoversCorpus(zwickau_corpus, london_corpus)

In [8]:
thesisCorpusStats.CorpusStats(burchard_corpus.corpus, burchard_corpus.name).get()

,paragraphs,Total words,Unique words,Avg. paragraph length (words),Avg. paragraph length (characters)
burchard_candidate_by_london_zwickau,217,17056,4334,78.599078,411.935484


In [9]:
thesisCorpusStats.CorpusStats(burchard_corpus.filter_short_p(), burchard_corpus.name).get()

,paragraphs,Total words,Unique words,Avg. paragraph length (words),Avg. paragraph length (characters)
burchard_candidate_by_london_zwickau,203,16799,4299,82.753695,433.566502


In [22]:
res = []
for i in zwickau_leftovers.corpus:
    res.append(len(i.split()))
print(res)

[4, 14, 2, 10, 13, 21, 98, 12, 8, 39, 46, 21, 231, 56, 15, 108, 23, 58, 28, 42, 28, 116, 10, 43, 26, 28, 5, 28, 18, 7, 23, 14, 26, 25, 25, 10, 50, 44, 79, 17, 10, 9, 19, 22, 145, 98, 45, 42, 18, 28, 7, 5, 101, 24, 31, 9, 5, 4, 38, 33, 38, 22, 17, 20, 41, 5, 22, 5, 23, 5, 150, 33, 83, 17, 39, 31, 54, 146, 103, 3, 9, 8, 21, 24, 38, 65, 34, 81, 46, 75, 57, 10, 13, 13, 9, 33, 6, 26, 70, 40, 14, 31, 55, 5, 37, 11, 43, 14, 26, 12, 32, 107, 19, 14, 11, 9, 9, 4, 21, 77, 71, 16, 21, 6, 3, 7, 23, 73, 3, 31, 31, 26, 43, 127, 93, 28, 14, 3, 25, 76, 16, 7, 8, 54, 41, 6, 79, 31, 9, 69, 143, 64, 63, 3, 30, 19, 6, 16, 41, 92, 13, 30, 79, 117, 32, 26, 16, 49, 38, 8, 9, 5, 21, 23, 8, 12, 11, 16, 7, 12, 20, 78, 11, 22, 33, 7, 6, 5, 38, 42, 3, 26, 3, 13, 4, 21, 29, 6, 25, 20, 91, 17, 6, 9, 29, 23, 25, 28, 32, 46, 93, 66, 14, 37, 25, 17, 16, 35, 57, 32, 43, 16, 64, 57, 7, 67, 43, 90, 20, 69, 19, 4, 8, 4, 1, 4, 5, 6, 6, 2, 3, 2, 0, 4, 1, 0, 4, 0, 0, 2, 5, 2, 2, 3, 4, 32, 35, 23, 6, 15, 14, 36, 20, 20, 26, 1

In [32]:
print(
    list(
        filter(lambda x: x <= 10, res)
    )
)

[4, 2, 10, 8, 10, 5, 7, 10, 10, 9, 7, 5, 9, 5, 4, 5, 5, 5, 3, 9, 8, 10, 9, 6, 5, 9, 9, 4, 6, 3, 7, 3, 3, 7, 8, 6, 9, 3, 6, 8, 9, 5, 8, 7, 7, 6, 5, 3, 3, 4, 6, 6, 9, 7, 4, 8, 4, 1, 4, 5, 6, 6, 2, 3, 2, 0, 4, 1, 0, 4, 0, 0, 2, 5, 2, 2, 3, 4, 6, 1, 7, 2, 6, 10, 8, 2, 6, 7, 7, 7, 4, 7, 5, 8, 5, 0, 4, 7, 6, 9, 9, 4, 8, 4, 9, 10, 10, 4, 9, 5, 8, 9]


In [35]:
featuresFactory  = thesisFactoryFeatures.FeaturesFactory(
    london_corpus = london_corpus,
    zwickau_corpus = zwickau_corpus
)

In [36]:
featuresFactory.london_VS_zwickau()

n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram


,index,corpus_version_label,a,a,a b,a be,a c,a ca,a ci,a cr,...,xulta,xum,xum,xum f,xum i,xum p,xus,xus,xus c,inner_mean_cosine_similarity_score
0,0.0,0.0,0.019250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.086334
1,1.0,0.0,0.010501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094742
2,2.0,0.0,0.009793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.114569
3,3.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111613
4,4.0,0.0,0.061078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.088173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,141.0,1.0,0.029025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.106153
299,142.0,1.0,0.027603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.102801
300,143.0,1.0,0.009834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.079071
301,144.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.077199
